# RAG Pipeline A/B Evaluation (Model-only)

This notebook loads your project and evaluates the **LLM-only** behavior of `rag.pipeline.RAGPipeline.answer` for a fixed set of queries, comparing two Ollama models (A/B).

In [1]:
import os

#OpenBLAS Warning : Detect OpenMP Loop and this application may hang...
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("GOTO_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")


# silence HF advisory warnings & progress bars
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()

# (optional) quiet sentence-transformers too
import logging
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)


/home/sinedo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# RAG ENV VARIABLE

In [2]:
import os

def init_env_defaults() -> None:
    s = os.environ.setdefault
    nproc = str(os.cpu_count() or 8)
    # ---- defaults (override via env) ----
    
    s("RAG_ART_INDEX_PATH", "src/st3405_data/index/st4305_text_bgem3.faiss")
    s("RAG_ART_STORE_PATH", "src/st3405_data/index/st4305_store.pkl.gz")
    s("RAG_CTX_BUDGET_CHARS", "10000")
    
    s("RAG_HYBRID_ALPHA", "0.5")      # weight for dense in fusion (0..1)
    s("RAG_QUERY_EXPAND", "1")        # 1=enable LLM query reformulation via Ollama
    s("RAG_QUERY_EXPAND_K", "0.5")    # keep as in script
    s("RAG_PRF_ENABLE", "1")
    s("RAG_PRF_TERMS", "8")
    s("RAG_OLLAMA_MODEL", "llama3:latest")
    s("RAG_RERANK_TIMEOUT", "15.0")
    s("RAG_SELECT_TIMEOUT", "10.0")
    s("RAG_QUERY_VARIATIONS", "10")

    s("OLLAMA_NO_GPU", "1")
    s("RAG_OLLAMA_NUM_PREDICT", "512")

    s("RAG_MIN_CTX", "2")
    s("RAG_MAX_CTX", "3")
    s("RAG_REFORM_CACHE_DISABLE", "1")

    s("RAG_CANDIDATE_K_FOR_RERANK", "15")
    s("RAG_FINAL_TOP_N", "3")

    s("RAG_DISABLE_SELECT", "1")

    # Models / batching
    s("RAG_EMBED_MAX_LENGTH", "256")
    s("RAG_EMBEDDER_MODEL", "BAAI/bge-m3")
    s("RAG_EMBED_BATCH", "1")

    #"BAAI/bge-reranker-base"
    # or jinaai/jina-reranker-v2-base-multilingual
    #naver/xprovence-reranker-bgem3-v1
    #naver/provence-reranker-debertav3-v1
    s("RAG_RERANKER_MODEL", "BAAI/bge-reranker-base")  
    
    # s("RAG_RERANKER_DEVICE", "cuda")
    # s("RAG_RERANKER_WINDOW", "384")
    # s("RAG_RERANKER_STRIDE", "256")
    # s("RAG_RERANKER_FP16", "1")
    # s("RAG_RERANKER_PAD_MAX", "0")
    # s("RAG_RERANKER_WINDOW_BATCH", "1")
    #cpu
    s("RAG_RERANKER_DEVICE", "cpu")
    s("RAG_RERANKER_WINDOW", "384")
    s("RAG_RERANKER_STRIDE", "256")
    s("RAG_RERANKER_FP16", "0")
    s("RAG_RERANKER_WINDOW_BATCH", "8")
    s("RAG_RERANKER_QUANTIZE", "1")  
    s("TOKENIZERS_PARALLELISM", "true")
    s("OMP_NUM_THREADS", nproc)
    s("MKL_NUM_THREADS", os.environ.get("OMP_NUM_THREADS", nproc))
    s("OPENBLAS_NUM_THREADS", os.environ.get("OMP_NUM_THREADS", nproc))
    s("NUMEXPR_NUM_THREADS", os.environ.get("OMP_NUM_THREADS", nproc))

    s("RAG_RERANKER_MAX_LEN", "384")
    s("RAG_RERANKER_AGG", "max")
    
    s("RAG_RERANKER_WINDOW_BATCH", "18")
    s("RAG_EMBED_DEVICE", "cpu")
    

    s("RAG_DISABLE_RERANK", "0")
    s("RAG_MAX_VARIANTS", "4")

    # Timeouts / perf
    s("RAG_EXPAND_TIMEOUT", "10.0")
    s("RAG_BM25_PARALLEL", "1")
    s("RAG_BM25_WORKERS", "16")

    s("RAG_DENSE_K_PER_QUERY", "10")
    s("RAG_SPARSE_K_PER_QUERY", "10")
    s("RAG_ENV_DUMPED", "0")
    s("RAG_FAISS_GPU", "0")

    # CUDA allocator
    s("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
    s("CUDA_VISIBLE_DEVICES", "1")
    
    s("KMP_AFFINITY", "granularity=fine,compact,1,0")

    try:
        import torch
        torch.set_grad_enabled(False)
        torch.set_num_threads(int(os.environ["OMP_NUM_THREADS"]))
        torch.set_num_interop_threads(1)
    except Exception:
        pass



# Chame no startup da sua app:
init_env_defaults()


In [3]:
import os
from pathlib import Path
import sys
from pathlib import Path

def get_root_path():
    """Always use the same, absolute (relative to root) paths

    which makes moving the notebooks around easier.
    """
    return Path(os.getcwd()).parent

PROJECT_DIR = Path(get_root_path())
assert PROJECT_DIR.exists(), PROJECT_DIR

if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

print("Using project at:", PROJECT_DIR)

Using project at: /mnt/dados/projetos/gps-tracking-rag-assistent-v1/src


In [4]:
def dump_env(prefix=None, redact=True, max_val_len=200):
    import os, re, sys
    """
    Print environment variables BEFORE the app starts.
    - prefixes: list[str] to include only certain prefixes; None = all
    - redact: hide secrets (KEY, TOKEN, SECRET, PASS, PWD)
    - max_val_len: clip long values for readability
    """
    secret_pat = re.compile(r"(KEY|SECRET|TOKEN|PASSWORD|PASS|PWD)", re.I)
    rows = []
    for k, v in os.environ.items():
        if prefix and not k.startswith(prefix):
            continue
        val = v
        if redact and secret_pat.search(k):
            val = "<redacted>"
        elif len(val) > max_val_len:
            val = val[:max_val_len] + "…"
        rows.append((k, val))
    for k, v in sorted(rows, key=lambda kv: kv[0]):
        print(f"[env] {k}={v}", file=sys.stderr, flush=True)

dump_env(prefix="RAG_")

[env] RAG_ART_INDEX_PATH=src/st3405_data/index/st4305_text_bgem3.faiss
[env] RAG_ART_STORE_PATH=src/st3405_data/index/st4305_store.pkl.gz
[env] RAG_BM25_PARALLEL=1
[env] RAG_BM25_WORKERS=16
[env] RAG_CANDIDATE_K_FOR_RERANK=15
[env] RAG_CTX_BUDGET_CHARS=10000
[env] RAG_DENSE_K_PER_QUERY=10
[env] RAG_DISABLE_RERANK=0
[env] RAG_DISABLE_SELECT=1
[env] RAG_EMBEDDER_MODEL=BAAI/bge-m3
[env] RAG_EMBED_BATCH=1
[env] RAG_EMBED_DEVICE=cpu
[env] RAG_EMBED_MAX_LENGTH=256
[env] RAG_ENV_DUMPED=0
[env] RAG_EXPAND_TIMEOUT=10.0
[env] RAG_FAISS_GPU=0
[env] RAG_FINAL_TOP_N=3
[env] RAG_HYBRID_ALPHA=0.5
[env] RAG_MAX_CTX=3
[env] RAG_MAX_VARIANTS=4
[env] RAG_MIN_CTX=2
[env] RAG_OLLAMA_MODEL=llama3:latest
[env] RAG_OLLAMA_NUM_PREDICT=512
[env] RAG_PRF_ENABLE=1
[env] RAG_PRF_TERMS=8
[env] RAG_QUERY_EXPAND=1
[env] RAG_QUERY_EXPAND_K=0.5
[env] RAG_QUERY_VARIATIONS=10
[env] RAG_REFORM_CACHE_DISABLE=1
[env] RAG_RERANKER_AGG=max
[env] RAG_RERANKER_DEVICE=cpu
[env] RAG_RERANKER_FP16=0
[env] RAG_RERANKER_MAX_LEN=384


# RagPipeline

In [5]:
# src/api/api_manager.py
from __future__ import annotations

import gzip
import os
import pickle
from pathlib import Path
from typing import List, Dict

import faiss
from rank_bm25 import BM25Okapi

# RAG plumbing (optimized service uses sentence selection + compact prompts)
from rag.config import RetrievalConfig
from rag.pipeline import RAGPipeline
from rag.config import RetrievalConfig, Timeouts, BuildLimits

# Rerankers / LLM / embedder

from rag.ollama import OllamaClient
from rag.init import get_embedder

# Tokenizer used to build/search BM25
from rag.utils import tokenize

# Retrievers
from rag.retrievers.dense_faiss import FaissRetriever
from rag.retrievers.sparse_bm25 import BM25Retriever
from rag.retrievers.multiquery_hybrid import MultiQueryHybridRetriever
from rag.query_expander import QueryExpander
from rag.types import DocStore

@staticmethod
def _env_int(name: str, default: int) -> int:
    try:
        return int(os.getenv(name, default))
    except Exception:
        return default

In [6]:
# from rag.ollama import OllamaClient
# from rag.locale_ptbr import SYSTEM_PROMPT, GENERATION_PROMPT
# client = OllamaClient()
# prompt = GENERATION_PROMPT.format(context="[1] Enable1: Ativa a Saída1", query="Qual o comando para habilitar a Saída 1")
# display(client.chat(prompt, temperature=0.0, system_prompt=SYSTEM_PROMPT))


In [ ]:
# Load FAISS index (GPU if available) ----
faiss_path = PROJECT_DIR / "st3405_data/index/st4305_text_bgem3.faiss"
if not Path(faiss_path).exists():
    raise FileNotFoundError(f"FAISS index not found at {faiss_path}")

index = faiss.read_index(str(faiss_path))
try:
    if os.getenv("RAG_FAISS_GPU", "0") == "1":
        if faiss.get_num_gpus() > 0:
            res = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(res, 0, index)
            index_device = "gpu"
except Exception:
    # GPU not available or FAISS GPU build not installed → stay on CPU
    index_device = "cpu"

# IVF/HNSW search breadth (no-op for Flat)
try:
    if hasattr(index, "nprobe"):
        index.nprobe = max(32, int(0.1 * getattr(index, "nlist", 100)))
    if hasattr(index, "hnsw"):
        index.hnsw.efSearch = int(os.getenv("RAG_FAISS_EFSEARCH", "256"))
except Exception:
    pass

# Load store: docs (text), meta, ids, and stored model name ----
store_path = PROJECT_DIR / "st3405_data/index/st4305_store.pkl.gz"
if not Path(store_path).exists():
    raise FileNotFoundError(f"Store file not found at {store_path}")

with gzip.open(store_path, "rb") as f:
    store = pickle.load(f)

docs: List[str] = store["docs"]
meta: List[dict] = store["meta"]
ids:  List[str] = store["ids"]

embedder_model_name = store.get("model", os.getenv("RAG_EMBEDDER_MODEL"))

assert index.ntotal == len(docs) == len(meta) == len(ids), (
    f"Index/store size mismatch: faiss={index.ntotal}, docs={len(docs)}, meta={len(meta)}, ids={len(ids)}"
)

# id -> metadata/text
id_to_meta = {ids[i]: {**meta[i], "doc_id": ids[i]} for i in range(len(ids))}
doc_store: DocStore = {ids[i]: docs[i] for i in range(len(ids))}

# ---- 4) Embedder (normalize=True for cosine/IP) ----
emb_model = get_embedder(embedder_model_name, use_fp16=True, normalize=True)

# ---- 5) BM25 over the docs using the SAME tokenizer ----
tokenized = [tokenize(t) for t in docs]
bm25 = BM25Okapi(
    tokenized,
    k1=float(os.getenv("RAG_BM25_K1", "1.4")),
    b=float(os.getenv("RAG_BM25_B", "0.4")),
)

# ---- 6) LLM client (Ollama) ----
ollama = OllamaClient(
    base_url=os.getenv("RAG_OLLAMA_BASE_URL", "http://localhost:11434"),
    model=os.getenv("RAG_OLLAMA_MODEL", "llama3:latest"),
    timeout=int(os.getenv("RAG_OLLAMA_TIMEOUT", "120")),
)

# ---- 7) Retrieval configuration ----
cfg = RetrievalConfig(
    query_variations=_env_int("RAG_QUERY_VARIATIONS", 5),
    dense_k_per_query=_env_int("RAG_DENSE_K_PER_QUERY", 80),
    sparse_k_per_query=_env_int("RAG_SPARSE_K_PER_QUERY", 80),
    rrf_k_const=_env_int("RAG_RRF_K_CONST", 50),
    candidate_k_for_rerank=_env_int("RAG_CANDIDATE_K_FOR_RERANK", 160),
    final_top_n=_env_int("RAG_FINAL_TOP_N", 12),
)

# ---- 8) Build retriever stack (dense + sparse → per-query hybrid → multi-query hybrid) ----
dense = FaissRetriever(faiss_index=index, id_map=ids, embedder=emb_model)
sparse = BM25Retriever(bm25=bm25, id_map=ids, tokenize_fn=tokenize)

use_expand = os.getenv("RAG_QUERY_EXPAND", "1") == "1"
expander = QueryExpander(llm=ollama) if use_expand else None

retriever = MultiQueryHybridRetriever(
    dense=dense,
    sparse=sparse,
    expander=expander,
    rrf_k=cfg.rrf_k_const,
    per_query_k=max(cfg.dense_k_per_query, cfg.sparse_k_per_query),
    final_limit=cfg.candidate_k_for_rerank,
)

# ---- 9) Reranker (LLM-based; swap to cross-encoder later if desired) ----
        
# if os.getenv("RAG_DISABLE_RERANK", "0") == "1":
#reranker = LLMReranker(llm=self.ollama, doc_store=self.doc_store)

# reranker_model = os.getenv("RAG_RERANKER_MODEL", "BAAI/bge-reranker-base")
# reranker = HFCrossEncoderLongReranker(
#     model_name=reranker_model,
#     doc_store=doc_store,
#     device=os.getenv("RAG_RERANKER_DEVICE", None),   # "cpu" | "cuda" | None
#     window_tokens=int(os.getenv("RAG_RERANKER_WINDOW", "512")),
#     stride_tokens=int(os.getenv("RAG_RERANKER_STRIDE", "384")),
#     agg=os.getenv("RAG_RERANKER_AGG", "max"),
#     fp16=os.getenv("RAG_RERANKER_FP16", "1") == "1",
#     pad_to_max=os.getenv("RAG_RERANKER_PAD_MAX", "0") == "1",
# )

# from rag.rerank.provence_reranker import ProvenceLongReranker
# reranker = ProvenceLongReranker(
#     model_name="naver/xprovence-reranker-bgem3-v1",
#     doc_store=doc_store,
#     max_length=8192,
#     device="cpu",
#     batch_size=2,
#     fp16=False,
#     pad_to_max=False,
# )
# ---- 10) Build the optimized RAG service ----

tmo = Timeouts()          # uses env defaults
limits = BuildLimits()    # uses env defaults

rag_pipeline = RAGPipeline(
    retriever=retriever,
    #reranker=reranker,
    embedder=emb_model,
    llm=ollama,
    doc_store=doc_store,
    cfg=cfg,
    tmo=tmo,
    limits=limits,
    meta_store=id_to_meta
)

In [8]:
# from rag.pruning_provence_x import ProvencePurePrunerx
# from rag.types import ScoredDoc

# top_docs = [
#         ScoredDoc(doc_id="20.envio de comandos", score=0.41, rank=0),      
# ]

# pruner = ProvencePurePrunerx(
#         model_name="naver/provence-reranker-debertav3-v1",
#         ctx_token_budget=1300,
#         device="cuda")

# context, stats = pruner.prune(
# query="Qual o comando para reiniciar o dispositivo via envio de comandos?",
# top_docs=top_docs, doc_store=doc_store)

# pruner.close()

In [9]:
os.environ.setdefault("RAG_LOG_ENABLED","1")

'1'

In [19]:
result = rag_pipeline.answer("Como consultar a versão do firmware(FW) do rastreador por comando?")
result["answer"]

[INFO] [MultiQueryHybridRetriever] start 
[INFO] [QueryExpander] started for query [Como consultar a versão do firmware(FW) do rastreador por comando?]
[INFO] query [Como consultar a versão do firmware(FW) do rastreador por comando?] reformulada. Variacoes [{
    "variantes_curtas": [
        "Firmware rastreador",
        "Versão FW",
        "Atualização FW",
        "Reset FW",
        "Reiniciar FW"
    ],
    "variantes_longas": [
        "Consultar a versão atual do firmware do rastreador de veículos",
        "Verificar a atualização disponível do firmware do rastreador",
        "Executar o comando de atualização do firmware do rastreador",
        "Reiniciar o firmware do rastreador para evitar problemas"
    ],
    "termos_exatos": [
        "reset",
        "reboot",
        "reiniciar",
        "atualizar",
        "actualizar",
        "update",
        "patch",
        "upgrade",
        "refresh",
        "restart"
    ],
    "siglas": [
        "FW",
        "RVD",
    

'CMD;4309999001;04;22;q'

In [11]:
result

{'answer': 'Requisição: CMD;4309999001;04;22;q\nResposta: RES;4309999001;04;22;T1,T2',
 'docs': ['3.funcionalidades', '8.serial rs232', '9.configuração de envio'],
 'prompt': "<CONTEXT>\n[1] 3. FUNCIONALIDADES  - Os rastreadores possuem uma solução híbrida de comunicação 4G com fallback para 2G, possui as funcionalidades ideais para auxiliar no rastreamento e gestão de frotas de forma versátil. - Acelerômetro - Zona de segurança através da função Towing Mode - Alerta de velocidade Alta / Baixa - Alerta de Falha de bateria (Externa e Backup) - 2 modos de baixo consumo (Sleep e Deep Sleep) - Detecção de Jammer - Envio de posição por Ângulo - Antifurto (ignição e porta) - Analise de motorista- DPA (acelerações, frenagens bruscas e curvas acentuadas) - LED para status de GPS e GPRS - Ignição física, virtual por tensão ou acelerômetro (movimento). - Horímetro (por ignição ligada) - Hodômetro (por GPS) - Interface 1-wire - Interface RS232  3. 1. ANTIFURTO IGNIÇÃO A função antifurto disponíve

In [12]:
# from transformers import AutoModel

# provence = AutoModel.from_pretrained("naver/provence-reranker-debertav3-v1", trust_remote_code=True)

# context = "Shepherd’s pie. History. In early cookery books, the dish was a means of using leftover roasted meat of any kind, and the pie dish was lined on the sides and bottom with mashed potato, as well as having a mashed potato crust on top. Variations and similar dishes. Other potato-topped pies include: The modern ”Cumberland pie” is a version with either beef or lamb and a layer of bread- crumbs and cheese on top. In medieval times, and modern-day Cumbria, the pastry crust had a filling of meat with fruits and spices.. In Quebec, a varia- tion on the cottage pie is called ”Paˆte ́ chinois”. It is made with ground beef on the bottom layer, canned corn in the middle, and mashed potato on top.. The ”shepherdess pie” is a vegetarian version made without meat, or a vegan version made without meat and dairy.. In the Netherlands, a very similar dish called ”philosopher’s stew” () often adds ingredients like beans, apples, prunes, or apple sauce.. In Brazil, a dish called in refers to the fact that a manioc puree hides a layer of sun-dried meat."
# question = 'What goes on the bottom of Shepherd’s pie?'

# provence_output = provence.process(question, context)
# print(f"Provence Output: {provence_output}")
# # Provence Output: {'reranking_score': 3.022725, pruned_context': 'In early cookery books, the dish was a means of using leftover roasted meat of any kind, and the pie dish was lined on the sides and bottom with mashed potato, as well as having a mashed potato crust on top.']]


In [13]:
# from transformers import AutoModel

# provence_pt = AutoModel.from_pretrained("naver/provence-reranker-debertav3-v1", trust_remote_code=True)

In [14]:
#provence_pt.to("cuda")

In [15]:
# context = "Shepherd’s pie. History. In early cookery books, the dish was a means of using leftover roasted meat of any kind, and the pie dish was lined on the sides and bottom with mashed potato, as well as having a mashed potato crust on top. Variations and similar dishes. Other potato-topped pies include: The modern ”Cumberland pie” is a version with either beef or lamb and a layer of bread- crumbs and cheese on top. In medieval times, and modern-day Cumbria, the pastry crust had a filling of meat with fruits and spices.. In Quebec, a varia- tion on the cottage pie is called ”Paˆte ́ chinois”. It is made with ground beef on the bottom layer, canned corn in the middle, and mashed potato on top.. The ”shepherdess pie” is a vegetarian version made without meat, or a vegan version made without meat and dairy.. In the Netherlands, a very similar dish called ”philosopher’s stew” () often adds ingredients like beans, apples, prunes, or apple sauce.. In Brazil, a dish called in refers to the fact that a manioc puree hides a layer of sun-dried meat."
# question = 'What goes on the bottom of Shepherd’s pie?'

# provence_output = provence.process(question, context)
# print(f"Provence Output: {provence_output}")

In [16]:
# import json
# from pathlib import Path
# import pandas as pd

# DATASET = Path(PROJECT_DIR / "st3405_data/st_4305_text.jsonl")

# rows = [json.loads(line) for line in DATASET.read_text(encoding="utf-8").splitlines()]
# df = pd.DataFrame(rows)

In [17]:
# rows[20]['text']

In [18]:
# provence_output = provence_pt.process(
#     "configurar o volume do microfone."
#     , "'20. ENVIO DE COMANDOS   20. 1. Lista de Comandos disponíveis  Preset: Apresenta as configurações atuais do equipamento. PresetP: Apresenta as configurações do parâmetro solicitado. ReqConMntSvr: Solicita que o dispositivo se reporte ao servidor de manutenção para verificar se há atualizações. ReqIMSI: Solicita o IMSI (International Mobile Subscriber Identity). ReqICCID: Solicita o ICCID do SimCard. ReqConNtw: Solicita em qual rede o dispositivo está conectado (2G, 4G). ReqOwnNo: Solicita o número da linha SimCard inserido, caso esteja cadastrado. SetGoogleMap: Configura o formato do link Google Maps. ReqGoogleMap: Solicita o formato do link Google Maps. StatusReq: Solicita a posição do dispositivo. Reset: Reinicia o dispositivo. Reboot: Reinicia o dispositivo. ReqVer: Solicita a versão do FW, utilizar o valor 1 no campo: Option.\nEnable1: Ativa a Saída1 Disable1: Desativa a Saída1 Enable2: Ativa a Saída2 Disable2: Desativa a Saída2 Enable3: Ativa a Saída3 Disable3: Desativa a Saída3 EraseAll: Apaga as posições que foram armazenadas na memória do dispositivo e desabilita a saída (caso esteja ativada) do equipamento. SetOdometer: Configura o odometro do dispositivo. InitMsgNo: Reinicia o contador das mensagens. SetHMeter: Configura o horímetro do dispositivo. InitCircleGeo: Inicia / apaga todas as cercas circulares no dispositivo. InitAllPolygonGeo: Inicia / apaga todas as cercas poligonais no dispositivo. Request Server Lock: Solicita o status da função Server Lock. ReqSttAssignmap: Solicita a configuração do Mapeamento das STT. ReqAltAssignmap: Solicita a configuração do Mapeamento das ALT. Encoding Type: Este comando configura o Encoding Type. O ID do dispositivo pode ser encriptado quando um comando é enviado por SMS. Favor observar a tabela abaixo: Valor Tipo de Controle de Saída 0 Não ativa nenhuma saída. 1 Ativa ambas Saídas 1 e Saída 2. 2 Ativa somente Saída 2. 3 Ativa somente Saída 1. GetDidOutCtrl: Este comando é para requisitar o tipo de controle de saída em relação ao ID do Motorista (iButton) configurado previamente. Set GeoFence Area Jamming: Ativa a função de detecção de Jammer dentro de uma área de Cerca configurada. 0 para “Desativar” e 1 para “Ativar”. Get GeoFence Area Jamming: Consulta se a função de detecção de Jammer dentro de uma área de Cerca está ativa ou não. 0 desativada, 1 ativada. Set Buzzer pulse off: Caso o Buzzer seja habilitado devido a uma situação de Velocidade Alta (Over Speed configurado), o Buzzer não irá pulsar caso essa opção esteja habilitada. 1 para “Habilitar” e 0 para “Desabilitar”. Get Buzzer pulse off: Consulta se a função “Buzzer pulse off” está habilitada ou não no equipamento.\nGet anti theft status: Consulta o status do serviço de antifurto. 0 desativado, 1 ativado. ReqAllDriver: Comando para solicitar todos os IDs de Motorista configurados no equipamento. ReqPolyinfo: Solicita as informações das cercas poligonais configuradas. No campo “Opção” inserir o ID da cerca a ser requisitado. Set immobilizer Always Pulsed: Configura o comportamento do pulso do imobilizador. 0 desativado, 1 ativado. Caso ativado, o imobilizar vai pulsar independente do status de ignição caso haja uma emergência por antifurto. Set Immob. Cycle time: Configura o tempo do ciclo (ativação) do imobilizador. Valores 0~7200 em segundos. Req Immob. Cycle time: Solicita o tempo do ciclo do imobilizador configurado previamente. Set immob speed limit: Quando o Imobilizador é ativado com o veículo em movimento, o Imobilizador irá ativar quando a velocidade do veículo estiver menor do que a velocidade configurada neste comando. Possíveis valores: 0 a 300 Km / h. InitIDPolygonGeo: Inicia / apaga a cerca poligonal especifica no dispositivo. É possível apagar as cercas poligonais dentro de um range de IDs específico. Exemplo: no campo “Opção” insira, “1,20”. Dessa forma, todas as cercas entre o ID 1 e 20 serão excluídas do equipamento. InitIDCircularGeo: Inicia / apaga a cerca circular especifica no dispositivo. É possível apagar as cercas circulares dentro de um range de IDs específico. Exemplo: no campo “Opção” insira, “1,500”. Dessa forma, todas as cercas entre o ID 1 e 500 serão excluídas do equipamento. ReqCircInfo: Solicita as informações das cercas circulares configuradas. No campo “Opção” inserir o ID da cerca a ser requisitado. ActivateAntiTheft: Ativa a função antifurto. Req circular ID: Solicita quais IDs estão configurados com cerca circular no equipamento, por grupo de IDs. Exemplo: No campo “Opção”, insira o valor “1”. Dessa forma, o equipamento reporta todos os IDs configurados no grupo 1 (IDs de 1 a 100). Grupo 1: Cerca eletrônica circular de ID 1 a 100 Grupo 2: Cerca eletrônica circular de ID 101 a 200 Grupo 3: Cerca eletrônica circular de ID 201 a 300 Grupo 10: Cerca eletrônica circular de ID 901~999. Start DPA Calibration: Inicia o modo de calibração da função DPA. Stop DPA Calibration: Finaliza o modo de calibração da função DPA.\nReqDPAParam: Solicita os parâmetros de DPA calibrados. ReqDPADefault: Solicita os parâmetros padrões do DPA. InitParkOdometer: Inicializa o odometro do dispositivo. InitAllDriverID: Este comando é utilizado para apagar todos os IDs de Motorista (iButton) configurados no equipamento. SetVol: Este comando é utilizado para configurar o volume do speaker. Valores validos: 0 (volume mínimo) a 6 (volume máximo). ReqVol: Este comando solicita o valor do volume do speaker. Valores validos: 0 (volume mínimo) a 6 (volume máximo). SetMicVol: Este comando é utilizado para configurar o volume do microfone. Valores validos: 0 (volume mínimo) a 15 (volume máximo). ReqMicVol: Este comando solicita o valor do volume do microfone. Valores validos: 0 (volume mínimo) a 6 (volume máximo).'")
# provence_output